# Homework 1 for CS 329P

**Authors**: YOUR_NAMES

**Emails**: YOUR_IDS@stanford.edu

**Submission.** Please insert your names and emails above, save your code in this notebook, and explain what you are doing along with your findings in text cells. You can think of it as a technical report with code. Before submission, please use `Kernel -> Restart & Run All` in the Jupyter menu to verify your code is runnable and save all outputs. Afterwards, you can either upload your raw notebook (`hw1.ipynb`) or an exported PDF version to the `Homework 1` assignment in Canvas. 


In this homework, we will train a house sales price predictor on the data we scraped previously. The purpose of this homework is to let you practice different techniques that you can use to preprocess raw data. Your job is to obtain the best root mean squared logarithmic error (RMSLE) on the test dataset. To make your job easy, we provide sample code to train a model to report RMSLE and a list of ideas you can explore.

**Note**: You can use either local runtimes to complete this assignment, or a hosted runtime (with GPU) on Colab. The second option generally runs faster. If using a local runtime, make sure that your Python version is less than 3.9 but at least 3.6, or you may have issues installing Autogluon. If using a runtime hosted on Colab, you can use the File Explorer pane on the left to upload the `house_sales.ftr` file. Make sure to wait until the file finishes uploading before running the next code block.

Additionally, if using a local runtime, please refer to the [AG document](https://auto.gluon.ai/stable/index.html#installation) for info on how to install autogluon. 

## Prepare Data 

Let's first read in the dataset we used in our [Exploratory Data Analysis (EDA)](https://c.d2l.ai/stanford-cs329p/_static/notebooks/cs329p_notebook_eda.slides.html). Note that we use the [`feather` format](https://arrow.apache.org/docs/python/feather.html), which is faster to read than CSV but uses more disk space. The file `home_sales.ftr` can be downloaded from the Assignments folder in Canvas.

Just for your information, it is generated with:

```python
data = pd.read_csv('house_sales.zip', dtype='unicode')
data.to_feather('house_sales.ftr')
```

The following code needs at least 2GB memory. If using a local runtime, please make sure your machine has enough memory. 

In [1]:
# Run the following line once to install. You may need to restart your runtime afterwards:
# !pip3 install numpy pandas autogluon mxnet --upgrade
import pandas as pd
import numpy as np
# print("NumPy 版本:", np.__version__)
data = pd.read_feather('../data/house_sales.ftr')


In [2]:
import scipy
import numpy as np
scipy.__version__, np.__version__

('1.11.2', '1.24.4')

We select a few common columns to make our training fast. You need to select more columns to make your model more accurate. 

In [3]:
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms']].copy()
# uncomment the below line to save memory
# del data

We copy the code from EDA to convert `Sold Price` to numerical values, which is our prediction target. We also remove examples whose prices are too high or too low.

In [4]:
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )]

/var/folders/zv/7_nb_lnx5llfxwncr2x_6tvh0000gn/T/ipykernel_37925/1326071105.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,c] = np.log10(


We use the house sales between 2021-2-15 and 2021-3-1 as our test data. You can use any example before 2021-2-15, but not after. In other words, we pretend we are launching our model on 2021-2-15 and testing it for 2 weeks. Here we only use sales in 2021 for fast training, but you can use more to improve accuracy. 

In [5]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1)
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

((24872, 6), (11510, 6))

Define our evaluation metric. 

In [6]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

## AutoGluon Baseline

We provide a baseline model trained by AutoGluon (AG). AG is an automl tool that performs automatic feature engineering, model selections, and ensemble. You are welcome to use any model and tool in achieving the best results possible in your homework. However, we recommend that you reuse the following training code so that you can focus on data preprocessing.

In [7]:
from autogluon.tabular import TabularPredictor
    
label = 'Sold Price'    
predictor = TabularPredictor(label=label).fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20230901_071844/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230901_071844/"
AutoGluon Version:  0.8.2
Python Version:     3.9.17
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:21:53 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6020
Disk Space Avail:   825.31 GB / 994.66 GB (83.0%)
Train Data Rows:    24872
Train Data Columns: 5
Label Column: Sold Price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7.546542675816042, 4.000043427276863, 5.75084, 0.39719)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using 

Test the performance of each model. 

In [8]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.264573,-0.287632,0.059081,0.014158,7.073444,0.059081,0.014158,7.073444,1,True,3
1,NeuralNetFastAI,-0.268273,-0.290594,0.086652,0.024421,13.651411,0.086652,0.024421,13.651411,1,True,8
2,LightGBMLarge,-0.268721,-0.288314,0.026853,0.007031,8.281847,0.026853,0.007031,8.281847,1,True,11
3,LightGBM,-0.270316,-0.288493,0.046828,0.009970,5.544618,0.046828,0.009970,5.544618,1,True,4
4,NeuralNetTorch,-0.273075,-0.291414,0.043024,0.010337,29.884430,0.043024,0.010337,29.884430,1,True,10
5,ExtraTreesMSE,-0.281389,-0.304345,0.187538,0.041746,1.131568,0.187538,0.041746,1.131568,1,True,7
6,WeightedEnsemble_L2,-0.288208,-0.284793,0.348478,0.068444,42.805013,0.001287,0.000157,0.091863,2,True,12
7,CatBoost,-0.304060,-0.286025,0.053306,0.005385,9.500134,0.053306,0.005385,9.500134,1,True,6
8,XGBoost,-0.325221,-0.288091,0.063323,0.010819,2.197018,0.063323,0.010819,2.197018,1,True,9
9,RandomForestMSE,-0.349723,-0.307923,0.172403,0.055255,2.928293,0.172403,0.055255,2.928293,1,True,5


Next, we compute the importance of each feature, along with several other metrics. It loooks like the `Sold On` feature is not very useful, likely because the houses in the test data were all sold late. You can choose to either remove such a feature, or find a way to extract a more useful presentation from it.

In [9]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 5 features using 5000 rows with 5 shuffle sets...
	9.4s	= Expected runtime (1.88s per shuffle set)
	4.29s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Bathrooms,0.077595,0.002865,2.224927e-07,5,0.083493,0.071697
Type,0.071763,0.005169,3.208633e-06,5,0.082407,0.061119
Year built,0.065390,0.002859,4.374602e-07,5,0.071277,0.059503
Bedrooms,0.012241,0.000821,2.413281e-06,5,0.013932,0.010551
Sold On,0.000174,0.000368,1.745692e-01,5,0.000932,-0.000584


Finally, let's predict and evaluate the RMSLE.

In [10]:
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])

0.2882079884279717

## Your Solution

Please include your solution in the following section. (You are welcome to edit and delete code in previous sections).

Your goal is to train a model using the features in the original dataset that minimizes the RMSLE on the validation dataset. While the naïve model achieves an RMSLE of ~0.3, it is possible to achieve an RMSLE of less than 0.08 on the same dataset.

Here is a list of ideas you could explore:

- More features: We only selected a small set of columns to use in training. You can add more, especially the ones we examined in EDA.
- Data type conversion: Most data columns are strings; you may need to convert them into numerical values.
- Data cleaning: There are NAN and outliers sprinkled throughout the dataset. You should find ways to selectively filter and remove them.
- More examples: We only included sales made in 2021; there is a large number of examples in previous years that you can also include.

In [11]:
# YOUR SOLUTION HERE

FIN